In [3]:
import pandas as pd

In [5]:
df = pd.read_csv("Cleaned_QVI_Data.csv")

In [7]:
df

,DATE,STORE_NBR,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PROD_WEIGHT,BRAND_NAME
0,2018-10-17,1,1000,YOUNG SINGLES/COUPLES,Premium,1,5,SeaSalt,2,6.0,175g,NCC
1,2018-09-16,1,1002,YOUNG SINGLES/COUPLES,Mainstream,2,58,Chikn&Garlic Aioli,1,2.7,150g,RRD
2,2019-03-07,1,1003,YOUNG FAMILIES,Budget,3,52,Sour Cream&Chives,1,3.6,210g,GrainWaves
3,2019-03-08,1,1003,YOUNG FAMILIES,Budget,4,106,Hony Soy Chckn,1,3.0,175g,NCC
4,2018-11-02,1,1004,OLDER SINGLES/COUPLES,Mainstream,5,96,Original Stacked Chips,1,1.9,160g,WW
...,...,...,...,...,...,...,...,...,...,...,...,...
264831,2018-12-08,88,2370701,YOUNG FAMILIES,Mainstream,240378,24,Sweet Chilli,2,7.2,210g,GrainWaves
264832,2018-10-01,88,2370751,YOUNG FAMILIES,Premium,240394,60,Tortilla ChpsFeta&Garlic,2,9.2,150g,Kettle
264833,2018-10-24,88,2370961,OLDER FAMILIES,Budget,240480,70,Crisps Lightly Salted,2,8.4,165g,Tyrrells
264834,2018-10-27,88,2370961,OLDER FAMILIES,Budget,240481,65,Salsa Dip Chnky Tom Ht,2,10.2,300g,OldElPaso


In [9]:
sales_per_store = df.groupby('STORE_NBR').agg({'TOT_SALES': 'sum'}).sort_values('TOT_SALES', ascending = False)
sales_per_store

,TOT_SALES
STORE_NBR,
226,18905.45
88,16333.25
165,15973.75
40,15559.50
237,15539.50
...,...
206,7.60
252,7.40
11,6.70


In [11]:
customers_per_store = df.groupby('STORE_NBR').agg({'LYLTY_CARD_NBR': 'nunique'}).sort_values('LYLTY_CARD_NBR', ascending = False)
customers_per_store

,LYLTY_CARD_NBR
STORE_NBR,
203,397
123,394
58,393
165,390
88,388
...,...
206,2
211,2
252,2


In [13]:
transactions_per_customer = df.groupby(['STORE_NBR', 'LYLTY_CARD_NBR']).agg({'TXN_ID': 'count'})

avg_transactions_per_cust = transactions_per_customer.groupby('STORE_NBR').agg({'TXN_ID': 'mean'})
avg_transactions_per_cust

,TXN_ID
STORE_NBR,
1,1.660870
2,1.616613
3,4.126374
4,4.427441
5,5.718487
...,...
268,1.639640
269,6.402390
270,6.707819


In [21]:
store_metrics = sales_per_store[['TOT_SALES']].join(customers_per_store[['LYLTY_CARD_NBR']])

In [25]:
store_metrics = store_metrics.join(avg_transactions_per_cust[['TXN_ID']])
store_metrics

,TOT_SALES,LYLTY_CARD_NBR,TXN_ID
STORE_NBR,,,
226,18905.45,382,5.293194
88,16333.25,388,4.827320
165,15973.75,390,4.664103
40,15559.50,388,4.546392
237,15539.50,382,4.672775
...,...,...,...
206,7.60,2,1.000000
252,7.40,2,1.000000
11,6.70,2,1.000000


In [33]:
store_metrics.rename(columns = {'LYLTY_CARD_NBR': 'No_of_customers', 'TXN_ID': 'No_of_transactions'}, inplace = True)

In [35]:
store_metrics

,TOT_SALES,No_of_customers,No_of_transactions
STORE_NBR,,,
226,18905.45,382,5.293194
88,16333.25,388,4.827320
165,15973.75,390,4.664103
40,15559.50,388,4.546392
237,15539.50,382,4.672775
...,...,...,...
206,7.60,2,1.000000
252,7.40,2,1.000000
11,6.70,2,1.000000


In [125]:
trial_stores = [77, 86, 88]

store_metrics_drop_trail = store_metrics.drop(trail_stores)

control_stores = {}

for trial in trial_stores:
    trial_data = store_metrics.loc[trial]  
    
    correlation = store_metrics_drop_trail.apply(lambda x: x.corr(trial_data), axis=1)
    
    correlation_sorted = correlation.sort_values(ascending=False)
    
    control_stores[trial] = correlation_sorted.head(1).index[0] 

print(control_stores)


{77: 50, 86: 70, 88: 155}


In [ ]:
#Using Pearson Correlation, we found the control stores for each of the trail stores. Now we can see whether the control stores performed well or not

In [133]:

comparison_results = {}

for trial in trial_stores:
    control_store = control_stores[trial]
    
    trial_sales = store_metrics.loc[trial, 'TOT_SALES']
    control_sales = store_metrics.loc[control_store, 'TOT_SALES']
    
    trial_customers = store_metrics.loc[trial, 'No_of_customers']
    control_customers = store_metrics.loc[control_store, 'No_of_customers']
    
    trial_transactions = store_metrics.loc[trial, 'No_of_transactions']
    control_transactions = store_metrics.loc[control_store, 'No_of_transactions']
    
    sales_difference = trial_sales - control_sales
    customers_difference = trial_customers - control_customers
    transactions_difference = trial_transactions - control_transactions
    
    comparison_results[trial] = {
        'Sales_difference': sales_difference,
        'Customers_difference': customers_difference,
        'Transactions_difference': transactions_difference
    }

comparison_df = pd.DataFrame(comparison_results).T

print("Comparison Results:\n", comparison_df)


Comparison Results:
     Sales_difference  Customers_difference  Transactions_difference
77             30.20                   0.0                -0.042135
86           1152.35                  29.0                -0.005645
88           5704.30                 133.0                -1.192288


In [135]:
#Based on the above metrics we can agree than trail stores performed better than control stores based on the total sales.
#However we need to consider one thing i.e., transactions wise control stores lagged but still able to generate better revenue.
#Is there any pricing difference which helped to generate better revenue even when there is a drop in transactions.
#I will suggest two things
#One is to check the store 77 which generated a very small difference in sales while other trail stores generated good revenue. I suggest to look if is 
#the trail store 77 was setup differently when compared to other stores.
#Although the revenue increased there is a drop in number of transactions. So need to see if the revenue increase is due to different pricing strategy 
#and need to come to a conclusion whther it is the expected outcome or not.